# Make BIDS root folder structure for a new EEG study 
(or new BIDS structure for an old study)

Creates the following empty folder structure for a new study:

```
— code
    - import
    - preproc
    - analysis

- derivatives
    - preproc_pipeline_1
        - s001
        - s002
    - preproc_pipeline_2
    - statistical_analysis_1

- rawdata
    - s001
    - s002
    
- README.md
    
- sourcedata
    - eeg
    - behav

```

- `code` contains all code used to operate on the data sets, including file conversion/import, preprocessing, and analysis
- `sourcedata` contains the original, unprocessed data files, organized into subfolders by modality (e.g., EEG, behavioural log files). These files should be write-protected as the original data should never be modified or deleted.
- `rawdata` contains BIDS-compliant raw datasets. These are typically the output of an import script that operates on data in `sourcedata` and converts it to BIDS-compliant formats.
- `derivatives` contain versions of the data that are processed by scripts in the `code` folder. Separate folders in `derivatives` should be made for different types of outputs, e.g., the output of a preprocessing pipeline would go in one folder, but a statistical analysis on that preprocessed data in another. This organization makes it easy to perform different batches of operations on data and compare the results, as well as keeping things organized. Within a `derivatives` subfolder, data should be organized by subject, potentially under subfolders for the outputs of different preprocessing steps, if these are saved.
- `README.md` is a Markdown file which you should edit to include a descirption of the study and experimental paradigm. This provides the context in which the data can be understood.  


In [35]:
import os
import os.path as op
import json
from mne_bids import make_dataset_description, update_sidecar_json, BIDSPath

## BIDS Project Parameters
Used to store metadata and for file/folder names

### Update this with your study info

In [36]:
# change to your study name
# IMPORTANT: when you collect data, you should use the study name, exactly as it's 
# specified here (same capitalization etc.), plus unique subject ID numbers
study_name = 'NCIL_EEG_template'

# Ethics board name and protocol ID
reb_info = 'Dalhousie SSHREB 20xx-xxxx'

# add additional project team member names
authors = ['Aaron J. Newman', ]

line_freq = 60 # Hz

instn = 'Dalhousie University'
instn_addr = 'Halifax, NS, Canada'
instn_dept = 'NeuroCognitive Imaging Lab, Department of Psychology & Neuroscience'

# Specify source of study funding ('NSERC', 'SSHRC', etc.)
funding = None 

license = 'BSD 3-clause'

## Run this cell to create directory structure

In [38]:
bids_root = 'data' 

os.mkdir(bids_root)

subdirs = [ 'code', op.join('code', 'import'), 'derivatives', 'rawdata', 'sourcedata', 
           op.join('sourcedata', 'eeg'), op.join('sourcedata', 'stim_log_files')
          ]    

if not op.isdir(code_path):
    # --- Create folder structure --- 
    for sd in subdirs:
        subd_path = op.join(bids_root, sd)
        os.mkdir(subd_path)   

    # --- Create metadata --- 
    make_dataset_description(path=bids_root,
                             name=study_name,
                             data_license=license,
                             authors=authors,
                             funding=funding,
                            )

    # --- Update metadata with additional info --- 
    entries = {'EthicsApprovals':reb_info,
               'InstitutionName':instn,
               'InstitutionAddress':instn_addr,
               'InstitutionalDepartmentName':instn_dept,
               'PowerLineFrequency':line_freq,
              }   
    sidecar_path = op.join(bids_root, 'dataset_description.json')
    with open(sidecar_path, 'r') as f:
        info = json.load(f)        
    info = {**info, **entries}
    with open(sidecar_path, 'w') as f:
        info = json.dump(info, f)
    
    # --- Write README file ---
    readme = ['# README - ' + study_name + '\n',
              'EEG study conducted in the [NeuroCognitive Imaging Lab](http://ncil.science), Department of Psychology & Neuroscience, [Dalhousie University](https://dal.ca). Directed by Aaron J Newman](https://aaronjnewman.com), [NeuroCognitive Imaging Lab](http://ncil.science).\n',
              '\n',
              'Code in this project largely relies on the Python pacakges MNE, mne-bids, mne-bids-pipeline. Aditional code written by Aaron J Newman and ---, released under the [BSD 3-clause license](https://opensource.org/licenses/BSD-3-Clause).\n',
              '\n',
              '## Study Description\n',
              '\n',
              '## Stimulus Presentation Details\n',
              ''
             ]

    with open(op.join(bids_root, 'README.md'), 'w') as f:
        f.writelines(readme)
        
else:
    print('Error: Target directory already exists')

Writing 'data/dataset_description.json'...
